<a href="https://colab.research.google.com/github/jlu1211/CSE163-Final-Project/blob/main/CSE_163_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [127]:
# Import all necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from matplotlib.projections.polar import RadialAxis
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
sns.set()

In [ ]:
# Data Cleaning
url = 'https://raw.githubusercontent.com/jlu1211/CSE163-Final-Project/main/Basketball_Shoes%20-%20Sheet1.csv?token=GHSAT0AAAAAACDICY72ZIJZXE5BECE6OLISZDVCBQA'
data = pd.read_csv(url)
data = data.drop(0)
data = data.reset_index(drop=True)
data_general = data.drop(['Heel-to-toe Offset', 'Recent Release', 'Asia Brand?', 'Budget Shoe? ($100)', 'Sizing Recommendation', 'Sizing Notes'], axis=1)
data_general['MSRP'] = data_general['MSRP'].str.replace('$', '').astype(int)
data_brand_avg = data_general.drop(['Year of Release', 'Weight per shoe (US9)'], axis=1).groupby('Brand').mean()
data_brand_avg

In [ ]:
# Research Question 1
def plot_scat_reg(data, x_col, y_col):
    data = data[['Brand', x_col, y_col]]
    top_brands = data['Brand'].value_counts().nlargest(4).index.tolist()
    data_top_brands = data[data['Brand'].isin(top_brands)]
    shoe_num = data_top_brands['Brand'].value_counts()

    palette = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

    # Create the scatter plot with colored points
    sns.scatterplot(x=x_col, y=y_col, hue='Brand', data=data_top_brands, s=50)

    # Regression Line and Correlation
    correlations = []
    for brand, color in zip(top_brands, palette):
        single_brand_data = data_top_brands[data_top_brands['Brand'] == brand]
        brand_trac_corr = single_brand_data[x_col].corr(single_brand_data[y_col])
        correlations.append(brand_trac_corr)
        sns.regplot(x=x_col, y=y_col, data=single_brand_data, ci=None, color=color)


    avg_corr = np.mean(correlations)
    legend_labels = [f'{brand} ({count}), Correlation: {corr:.2f}' for brand, count, corr in zip(shoe_num.index, shoe_num.values, correlations)]
    legend_labels.append(f'Average Correlation: {avg_corr:.2f}')
    legend_handles = [plt.Line2D([], [], color=color, marker='o', linestyle='', markersize=8) for color in palette]
    legend_handles.append(plt.Line2D([], [], color='black', marker='o', linestyle='', markersize=8))
    plt.title(f'{x_col} vs {y_col} for Top {4} Brands')
    plt.xlabel(x_col)
    plt.ylabel(y_col)

    plt.legend(title='Brand', bbox_to_anchor=(1.05, 1), loc='best', labels=legend_labels, handles=legend_handles)
    plt.show()

def plot_attributes_vs_overall(data):
  attributes = data.columns[2:10]
  for attribute in attributes:
    plot_scat_reg(data, attribute, 'Overall Rating')
plot_attributes_vs_overall(data_general)

In [ ]:
# Plot all brands radar by their attributes
attributes = data_brand_avg.columns[0:8]
brands = data_brand_avg.index.tolist()

def radar_plot_trac_vent(data, brands):
  for brand in range(len(brands)):
    plot_radar(data, brand)


def plot_radar(data, brand_index):
  attribute_data = data.iloc[brand_index, 0:8].values.tolist()
  fig = go.Figure(data=go.Scatterpolar(
      r=attribute_data,
      theta=attributes,
      name=data.index[brand_index],
      fill='toself'
  ))

  fig.update_layout(
      polar=dict(
          radialaxis=dict(
              visible=True
          ),
      ),
      showlegend=True
  )
  fig.show()

radar_plot_trac_vent(data_brand_avg, brands)

In [ ]:
# Radar plot that compare four brands attributes by comparing back to back
def plot_two_brand_radar(data, brands, attributes):
  for brand in brands:
    print(brand)
    compare_top_four_brands(data, brand, attributes)

def compare_top_four_brands(data, brands, attributes):
    brand1 = brands[0]
    brand2 = brands[1]
    brand1_index = data.index.get_loc(brand1)
    brand2_index = data.index.get_loc(brand2)

    fig = go.Figure()

    fig.add_trace(go.Scatterpolar(
        r=data.iloc[brand1_index, 0:8].values.tolist(),
        theta=attributes,
        fill='toself',
        name=brand1
    ))
    fig.add_trace(go.Scatterpolar(
        r=data.iloc[brand2_index, 0:8].values.tolist(),
        theta=attributes,
        fill='toself',
        name=brand2
    ))
    fig.update_layout(
      polar=dict(
        radialaxis=dict(
          visible=True,
        )),
      showlegend=True
    )

    fig.show()

top_four_brands = [['NIKE', 'ADIDAS'], ['NIKE', 'LI-NING'], ['NIKE', 'JORDAN'],
               ['ADIDAS', 'LI-NING'], ['ADIDAS', 'JORDAN'], ['LI-NING', 'JORDAN']]
attributes = data_brand_avg.columns[0:8]
plot_two_brand_radar(data_brand_avg, top_four_brands, attributes)